## Import R libraries

In [ ]:
library(raster)
library(rgdal)
library(randomForest)
library(ggplot2)
library(sf)

## Load Satellite Image (Raster Data)

In [ ]:
raster_data <- stack("data/landcover.tif")

## Calculate NDVI

In [ ]:
ndvi <- (raster_data[[4]] - raster_data[[1]]) / (raster_data[[4]] + raster_data[[1]])

## Plot NDVI

In [ ]:
ggplot() +
  geom_raster(data = as.data.frame(rasterToPoints(ndvi)), aes(x = x, y = y, fill = layer)) +
  scale_fill_gradient(low = "yellow", high = "green") +
  ggtitle("NDVI Map")
ggsave("outputs/ndvi_map.png")

## Load Training Data

In [ ]:
train_data <- st_read("data/training_points.geojson")
X <- train_data[, c("nir", "red")]
y <- as.factor(train_data$class)

## Train/Test Split

In [ ]:
set.seed(42)
train_idx <- sample(1:nrow(X), size = 0.8 * nrow(X))
X_train <- X[train_idx, ]
y_train <- y[train_idx]
X_test <- X[-train_idx, ]
y_test <- y[-train_idx]

## Train Random Forest Model

In [ ]:
rf_model <- randomForest(X_train, y_train, ntree = 100, set.seed(42))

## Save Model

In [ ]:
saveRDS(rf_model, "models/rf_model.rds")

## Evaluate Model

In [ ]:
preds <- predict(rf_model, X_test)
accuracy <- sum(preds == y_test) / length(y_test)
print(paste("Model Accuracy:", round(accuracy, 2)))